In [150]:
import os
import pandas as pd
import numpy as np

### Ler dataframes

#### l____________________________________________________________________________________________________________________

In [210]:
gt = pd.read_csv('../data/groundtruth.csv',sep=';')

In [211]:
pc='../../fdetector/outputs/concat/'
pn='../../fdetector/outputs/all/'

def readcsvs(dire):
    o = pd.DataFrame()
    for f in os.listdir(dire):
        if f.split('.')[-1]=='csv':
            o = pd.concat( (o, pd.read_csv(os.path.join(dire,f))),axis=0)
    return o

oc = readcsvs(pc)
on = readcsvs(pn)
o = pd.concat( (oc,on), axis=0)

### Checagem

In [212]:
#Deve retornar vazio
o[~o['Nome'].isin(gt['Ponto de medicao'])]

,Unnamed: 0,Nome,Mês,Dia_leitura,Valores faltantes_mm,leituras faltantes_mm,valores_faltantes_fasorial,leituras_faltantes_fasorial,colunas_inconsistentes_fasorial,Perfil cadastrado,...,tensão mínima (100) - dias,tensão mínima (100) - minutos,corrente zero - dias,corrente zero -minutos,corrente negativa - dias,corrente negativa -minutos,potência negativa - dias,potência negativa - minutos,anomalia de ângulos - dias,anomalia de ângulos - minutos


#### l____________________________________________________________________________________________________________________

### Selecionar abordagem
<p>
    <li> Abordagem 1: 'NADA APURADO' => 0 ,restante 1 (porém conserva rótulos para observação)
    <li> Abodagem 2: 1 para Fraude ou defeito com perda.
    <li> Abordagem 3: 1 apenas para fraude
</p>

In [362]:
global mapgt
mapgt= [
{'NADA APURADO':0, 'LOCAL VAZIO/ DESATIVADO':1, 'CLIENTE DESLIGADO OU CORTADO':2, 'DEFEITO SEM PERDA':3, 'DEFEITO GEROU PERDA':4 ,  'FRAUDE CONSTATADA':5},
{'NADA APURADO':0, 'LOCAL VAZIO/ DESATIVADO':0, 'CLIENTE DESLIGADO OU CORTADO':0, 'DEFEITO SEM PERDA':0, 'DEFEITO GEROU PERDA':1 ,  'FRAUDE CONSTATADA':1},
{'NADA APURADO':0, 'LOCAL VAZIO/ DESATIVADO':0, 'CLIENTE DESLIGADO OU CORTADO':0, 'DEFEITO SEM PERDA':0, 'DEFEITO GEROU PERDA':0 ,  'FRAUDE CONSTATADA':1}
]    


##### Escolher forma de selecionar os diagnśoticos

In [363]:

def thresh(d,t):
    d2 = d.copy()
    for i in range(0,t):
        if i in d2:
            d2[i]=0
    return d2


Escolher diag de 2 a 4

In [370]:
diag=0
global diags0
diags0={0:0, 1:1, 2:2, 3:3, 4:4}
diags=thresh(diags0,diag)
diags

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4}

##### Selecionar abordagem abaixo (0,1 ou 2)

In [371]:
abordagem = 0
labels = mapgt[abordagem]

### Preparar datasets

In [375]:
o2, gt2 = o.copy(), gt.copy()
o2=o2.set_index('Nome').loc[:,'DIAGNÓSTICO-FINAL'].groupby(level=0).max().sort_index()
gt2 = gt2.set_index('Ponto de medicao').sort_index().loc[:,'Retorno de campo'].map(labels)
gtd=gt2.to_dict()

#criar dfo que tem o groundtruth e o diagnostico. Eliminar nan (9) pode ser qualquer valor.
dfo = pd.DataFrame(o2)
dfo.insert(loc=0,value=dfo.index.map(gtd), column='gt')
dfo.fillna(value=9,inplace=True)
dfo=dfo[dfo['gt']!=9]
dfo.columns = ['groundtruth', 'diagnostico_cas']
dfo['diagnostico_cas']=dfo['diagnostico_cas'].map(diags)

#Deixar 0 e 1 para fazer análise com duas classses apenas
dfo1=dfo.applymap(lambda x: 1 if x>0 else 0)

#valores que 'batem' serão 1. Senão 0.
dfo1.insert(loc=0, column='acerto', value=dfo1[['groundtruth','diagnostico_cas']].apply((lambda x: 1 if x[0]==x[1] else 0),axis=1))

### Resultado

In [376]:
print('acertos ={} \n {:.1f}%'.format(dfo1['acerto'].sum(), dfo1['acerto'].sum()/len(dfo1)*100))

acertos =195 
 91.1%


### Instalações não detectadas

In [381]:
# visualizar instalações com erro
erros=dfo1[dfo1['acerto']==0].index
dferros = dfo.loc[erros,:]
dferros

,groundtruth,diagnostico_cas
Nome,,
BT2A410102348,0.0,2
BT2A412948154,5.0,0
BTI400099015,3.0,0
BTI400355240,0.0,1
BTI410166160,4.0,0
BTI410458978,4.0,0
BTI410722548,4.0,0
BTI411040881,0.0,4
BTI411655117,0.0,2


#### Falsos negativos

In [383]:
fn = dferros[dferros['groundtruth']==0]
fn

,groundtruth,diagnostico_cas
Nome,,
BT2A410102348,0.0,2
BTI400355240,0.0,1
BTI411040881,0.0,4
BTI411655117,0.0,2
BTI412305424,0.0,3
BTI414295413,0.0,4
BTI420070269,0.0,2
BTI420410037,0.0,3
BTI420429197,0.0,3


#### Falsos positivos

In [384]:
fp = dferros[dferros['groundtruth']!=0]
fp

,groundtruth,diagnostico_cas
Nome,,
BT2A412948154,5.0,0
BTI400099015,3.0,0
BTI410166160,4.0,0
BTI410458978,4.0,0
BTI410722548,4.0,0
BTI411981177,4.0,0
BTI412506733,4.0,0
BTI414239708,4.0,0
BTI421158453,4.0,0
